In [ ]:
# data visualizations

water_alpha = np.polyfit(temps_water, mu_mtx_water, 1)[0]
c50_alpha = np.polyfit(temps_c50, mu_mtx_c50, 1)[0]
c300_alpha = np.polyfit(temps_c300, mu_mtx_c300, 1)[0]
c600_alpha = np.polyfit(temps_c600, mu_mtx_c600, 1)[0]
protein_alpha = np.polyfit(temps_protein, mu_mtx_protein, 1)[0]
all_slopes = [water_alpha, c50_alpha, c300_alpha, c600_alpha, protein_alpha]
legend = ["water", "c50", "c300", "c600", "protein"]

# plot energy-specific thermal sensitivities for each material
plt.figure(figsize=(10,10))
for i, material in enumerate(all_slopes): 
    plt.scatter((i+1)*np.ones(4), material, label = legend[i])

    for k, label in enumerate(SpectralScan.energies):
        plt.annotate(label, (i+1, material[k]))
        
plt.legend()
plt.xlabel("index")
plt.ylabel("thermal sensitivity (mu/C)")
plt.show()

# plot attenuation vs. temperature for a given energy channel
channel = 3
plt.plot(temps_water, mu_mtx_water[:,channel], label="water")
plt.plot(temps_c50, mu_mtx_c50[:,channel], label="50")
plt.plot(temps_c300, mu_mtx_c300[:,channel], label="300")
plt.plot(temps_c600, mu_mtx_c600[:,channel], label="600")
plt.xlabel("temperature (C)")
plt.ylabel("attenuation (1/cm)")
plt.legend()
plt.show()

# plot attenuation vs. energy for a given temperature
temp = 3
plt.plot(SpectralScan.energies, mu_mtx_water[temp,:], label="water")
plt.plot(SpectralScan.energies, mu_mtx_c50[temp,:], label="50")
plt.plot(SpectralScan.energies, mu_mtx_c300[temp,:], label="300")
plt.plot(SpectralScan.energies, mu_mtx_c600[temp,:], label="600")
plt.xlabel("energy (kvp)")
plt.ylabel("attenuation (1/cm)")
plt.legend()
plt.show()

temp = 0 
plt.plot(SpectralScan.energies, mu_mtx_cacl2_600mM[temp,:], label="600")
plt.plot(SpectralScan.energies, mu_mtx_c300[temp,:], label="300")
plt.plot(SpectralScan.energies, mu_mtx_c50[temp,:], label="50")
plt.plot(SpectralScan.energies, mu_mtx_water[temp,:], label="water")
plt.plot(SpectralScan.energies, mu_mtx_protein_shake[temp,:], label="protein shake")
plt.xlabel("energy (kvp)")
plt.ylabel("attenuation (1/cm)")
plt.legend()
plt.show()

In [ ]:
""" ----- TEMPERATURE PREDICTION CLASS ----- """
# using one-step algorithm described in https://arxiv.org/abs/2202.13297
class TemperaturePrediction():
    def __init__(self, basis1_mu, basis2_mu, basis1_alpha, basis2_alpha):
        # all of these should be column vectors
        self.basis1_mu = basis1_mu
        self.basis2_mu = basis2_mu
        self.basis1_alpha = basis1_alpha
        self.basis2_alpha = basis2_alpha
        self.attenuations = None
 
    def simulate_attenuations(self, v1, v2, delta_t):
        mu = v1*(self.basis1_mu + delta_t*self.basis1_alpha) + v2*(self.basis2_mu + delta_t*self.basis2_alpha)
        return mu
 
    def predict_temperature(self, attenuations):
        mtx_cols = [self.basis1_mu, self.basis2_mu, self.basis1_alpha, self.basis2_alpha]
 
        # check that all are column vectors
        for col in mtx_cols:
            assert(col.shape[1] == 1)
 
        mtx = np.concatenate(mtx_cols, axis=1)
        row_4 = np.array([[1, 1, 0, 0]]) # v1 + v2 = 1
        mtx = np.concatenate([mtx, row_4], axis=0)
        mtx = np.delete(mtx, 1, 0) # delete row 1 which corresponds to 33-45 kvp bin
        inverse = np.linalg.inv(mtx)
 
        print(attenuations)
        print("")
        print(mtx)
        print("")
 
        result = np.matmul(inverse, attenuations)
        return result
 
water_cacl2_temp_predictor = TemperaturePrediction(
    np.expand_dims(mu_mtx_water[0,:], axis=1), # at 33 C
    np.expand_dims(mu_mtx_c300[0,:], axis=1),
    np.expand_dims(water_alpha, axis=1),
    np.expand_dims(c300_alpha, axis=1),
)
 
# thermal sensitivity as a weighted average is not a good assumption
for i in range(5):
    mu = np.reshape(mu_mtx_c50[i,:], (4,1)) # read in attenuation at each temperature
    mu = np.delete(mu, 1, 0) # delete row 1 which corresponds to 33-45 kvp bin
    mu = np.concatenate([mu, np.array([[1]])], axis=0)

    pred = water_cacl2_temp_predictor.predict_temperature(mu)
    print(f"v1 = {pred[0]}")
    print(f"v2 = {pred[1]}")
    print(f"t1 = {pred[2] / pred[0] + 33}")
    print(f"t2 = {pred[3] / pred[1] + 33}")
    print(f"t = {temps_c300[i]}")
    print("--------------------")